In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import json
import re
import requests
import time
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

In [10]:
superbowl = pd.read_csv('..\\data\\nfl_halftime_wide.csv')
superbowl=superbowl['Date']
superbowl = pd.to_datetime(superbowl)

## Sign in to spotify

In [10]:
soups=[]

options = Options()
options.binary_location = "C:\Program Files\chrome-win64\chrome.exe"
driver=webdriver.Chrome(chrome_options=options,executable_path="C:\Program Files\chromedriver-win64\chromedriver.exe")

driver.get("https://accounts.spotify.com/en/login?continue=https%3A%2F%2Fcharts.spotify.com/login")
username = driver.find_element_by_id("login-username")
password = driver.find_element_by_id("login-password")

username.send_keys("username")
password.send_keys("password")

driver.find_element_by_id("login-button").click()



c:\Users\bisho\.conda\envs\IDA\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


## Scrape spotify charts

In [81]:
song_soups=[]

for sb in superbowl[-8:]:
    driver.get('https://charts.spotify.com/charts/view/regional-us-weekly/'+str(sb.year)+'-'+str(sb.month).zfill(2)+'-'+str(sb.day+4).zfill(2))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    page_source = driver.page_source

    song_soups.append(bs(page_source, 'html.parser'))

In [74]:
soups=[]

for sb in superbowl[-3:]:
    driver.get('https://charts.spotify.com/charts/view/artist-us-weekly/'+str(sb.year)+'-'+str(sb.month).zfill(2)+'-'+str(sb.day+4).zfill(2))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    page_source = driver.page_source

    soups.append(bs(page_source, 'html.parser'))

## Parse soup and save results

In [75]:
## Parse artist html soup

rank=[]
name = []
peak = []
previous = []
streak = []
year = []

yr=0
for soup in soups:
    i=0
    for row in soup.find_all('tr'):
        if i ==1:    
            rank.append(row.find('span', attrs={'aria-label':"Current position"}))
            name.append(row.find('span', {"class":"styled__StyledTruncatedTitle-sc-135veyd-22 kKOJRc"}))
            peak.append(row.find('td', {"class":"TableCell__TableCellElement-sc-1nn7cfv-0 dLdEGj styled__RightTableCell-sc-135veyd-4 kGfYTK"}))
            previous.append(row.find('td', {"class":"TableCell__TableCellElement-sc-1nn7cfv-0 dLdEGj styled__RightTableCell-sc-135veyd-4 kGfYTK"}))
            streak.append(row.find('td', {"class":"TableCell__TableCellElement-sc-1nn7cfv-0 dLdEGj styled__RightTableCell-sc-135veyd-4 kGfYTK"}))
            year.append(2022+yr)
        else:
            i=1
    
    yr+=1
        


In [ ]:
artists= pd.DataFrame({'year':year,'rank':rank,'name':name,'peak': peak, 'previous':previous, 'streak':streak})

In [78]:
for col in artists.columns:
    if col != 'year':
        artists[col] = artists[col].apply(lambda x: x.text)

In [36]:
artists.to_csv('..\\data\\artists.csv', index=False)

In [82]:
## parse html soup

rank=[]
name = []
peak = []
previous = []
streak = []
year = []
title=[]
streams = []

yr=0
for soup in song_soups:
    i=0
    for row in soup.find_all('tr'):
        if i ==1:    
            rank.append(row.find('span', attrs={'aria-label':"Current position"}))
            name.append(row.find('a', {'class':"styled__StyledHyperlink-sc-135veyd-25 bVVLJU"}))
            title.append(row.find('span', {"class":"styled__StyledTruncatedTitle-sc-135veyd-22 kKOJRc"}))
            peak.append(row.find('td', {"class":"TableCell__TableCellElement-sc-1nn7cfv-0 dLdEGj styled__RightTableCell-sc-135veyd-4 kGfYTK"}))
            previous.append(row.find('td', {"class":"TableCell__TableCellElement-sc-1nn7cfv-0 dLdEGj styled__RightTableCell-sc-135veyd-4 kGfYTK"}))
            streak.append(row.find('td', {"class":"TableCell__TableCellElement-sc-1nn7cfv-0 dLdEGj styled__RightTableCell-sc-135veyd-4 kGfYTK"}))
            streams.append(row.find('td', {"class":"TableCell__TableCellElement-sc-1nn7cfv-0 kJgiFu styled__RightTableCell-sc-135veyd-4 kGfYTK"}))
            year.append(2017+yr)
        else:
            i=1
    
    yr+=1

In [83]:
songs = pd.DataFrame({'year':year,'rank':rank,'song':title,'name':name,'peak': peak, 'previous':previous, 'streak':streak, 'streams':streams})

In [84]:
for col in songs.columns:
    if col != 'year':
        songs[col] = songs[col].apply(lambda x: x.text)


In [34]:
songs.to_csv('..\\data\\songs.csv',index=False)

## Scrape UK charts

In [34]:
## no data prior to 2016
soups=[]


for sb in superbowl[-9:]:
    page = requests.get('https://www.officialcharts.com/charts/albums-sales-chart/'+str(sb.year)+str(sb.month)+str(sb.day)+'/7511/')
    soup = bs(page.content, 'html.parser')  
    soups.append(soup)


In [37]:
## Parse the data

rank = []
title = []
artist = []
previous = []
peak = []
weeks = []
year = []
i=0
for soup in soups:
    year.extend([2016+i]*100)
    for row in soup.find_all('div', {'class':'chart-item-content relative flex w-full'}):
        rank.append(row.find('div', {'class':'position'}).text[7:])
        t = (row.find('a', {'class':'chart-name font-bold inline-block'}).text)
        if t[:3]=='New':
            title.append(t[3:])
        else:
            title.append(t)
        artist.append(row.find('a', {'class':'chart-artist text-lg inline-block'}).text)
        previous.append(row.find('li', {'class':'movement px-2 py-1 rounded-md inline-block mr-1 sm:mr-2'}).text[4:].replace(',',''))
        peak.append(row.find('li', {'class':'peak px-2 py-1 rounded-md inline-block mr-1 sm:mr-2'}).text[6:].replace(',',''))
        weeks.append(row.find('li', {'class':'weeks px-2 py-1 rounded-md inline-block mr-1 sm:mr-2'}).text[7:].replace(',',''))
    i+=1
    


### Save data to dataframe and csv

In [38]:
uk_charts = pd.DataFrame({'year':year,'rank':rank,'title':title,'artist':artist,'previous':previous,'peak':peak,'weeks':weeks})

In [40]:
uk_charts.to_csv('..\\data\\uk_charts.csv', index=False)

## Scrape US Charts

In [ ]:
pages = pd.read_csv('..\\references\\wayback.csv')
soups={}

for iter, row in pages.iterrows():
    page = requests.get(row['link'])
    soups[row['date']]=bs(page.content, 'html.parser')

### Save scraped web data as html to parse later

In [ ]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

paths = []
for key in soups.keys():
    k=key.replace('/','_')
    path = '..\\references\\html\\'+k+'.html'
    save_html(bs.encode(soups[key]), path)
    paths.append(path)

pages['path']=paths
pages.to_csv('..\\references\\wayback.csv', index=False)

### Load scraped html

In [38]:
pages = pd.read_csv('..\\references\\wayback.csv')
soups = {}
for index, row in pages.iterrows():
    with open(row['path'], 'rb') as f:
        soups[row['date']] = bs(f, 'html.parser')


### Parse webpage content

In [39]:
previous = []
rank = []
artist = []
label = []
total = []
change = []
albums = []
date=[]
tea = []
sea = []
switch = False

for key in soups.keys():
    soup = soups[key]
    date.extend([key]*50)
    if key == "2/1/2019":
        switch = True
    if switch:
        
        for rows in soup.find_all('tr', {'class':['hits_album_chart_header_full_alt2','hits_album_chart_header_full_alt1',]}):
            previous.append(rows.find('td', {'class':'hits_album_chart_full_tw'}))
            rank.append(rows.find('td', {'class':'hits_album_chart_full_lw'}))
            artist.append(rows.find('span', {'class':'hits_album_chart_item_details_full_artist'}))
            label.append(rows.find('span', {'class':'hits_album_chart_item_details_full_label'}))
            total.append(rows.find('td', {'class':['hits_album_chart_item_top_details_full_sales chart_tweak col_sales','hits_album_chart_item_details_full_sales chart_tweak col_sales']}))
            change.append(rows.find('td', {'class':['hits_album_chart_item_top_details_full_change chart_tweak col_change','hits_album_chart_item_details_full_change chart_tweak col_change']}))
            albums.append(rows.find('td', {'class':['hits_album_chart_item_top_details_full_sales_albums chart_tweak col_albums','hits_album_chart_item_details_full_sales_albums chart_tweak col_albums']}))
            tea.append(rows.find('td', {'class':['hits_album_chart_item_top_details_full_sales_tea chart_tweak col_tea','hits_album_chart_item_details_full_sales_tea chart_tweak col_tea']}))
            sea.append(rows.find('td', {'class':['hits_album_chart_item_top_details_full_sales_sea chart_tweak col_sea','hits_album_chart_item_details_full_sales_sea chart_tweak col_sea']}))

    

    else:
        for rows in soup.find_all('tr', {'class':['hits_album_chart_header_full_alt2','hits_album_chart_header_full_alt1',]}):
            previous.append(rows.find('td', {'class':['hits_album_chart_full_tw','hits_album_chart_tw_full_top']}))
            rank.append(rows.find('td', {'class':['hits_album_chart_full_lw','hits_album_chart_lw_full_top']}))
            artist.append(rows.find('span', {'class':['hits_album_chart_item_details_full_artist','hits_album_chart_item_top_full_details_artist']}))
            label.append(rows.find('td', {'class':'hits_album_chart_item_top_details_full_label'}))
            total.append(rows.find('td', {'class':'hits_album_chart_item_top_details_full_sales'}))
            change.append(rows.find('td', {'class':'hits_album_chart_item_top_details_full_change'}))
            albums.append(None)
            tea.append(None)
            sea.append(None)

         
        

### Save parsed data as csv and dataframe

In [40]:
us_charts = pd.DataFrame({'date':date,'rank':rank,'lw':previous,'artist':artist,'label':label,'total':total,'change':change,'albums':albums,'tea':tea,'sea':sea})

In [32]:
us_charts.to_csv('..\\data\\hdd_charts.csv',index=False)